# Controle Sintético: Impacto da Fala do Ministro Haddad sobre o BRL

## Objetivo

Estimou-se o impacto da declaração do Ministro Fernando Haddad em 27/11/2024 — referente à proposta de isenção de IRPF para trabalhadores com salário de até R$ 5.000 — sobre a taxa de câmbio brasileira (USD/BRL).

Para isolar o componente doméstico do choque cambial do movimento global das moedas emergentes, aplicou-se a metodologia de **Controle Sintético**: construiu-se um 'BRL hipotético' a partir de uma cesta ponderada de moedas emergentes comparáveis. O desvio entre o BRL real e o BRL sintético após a data do evento é interpretado como o efeito atribuível exclusivamente ao choque de natureza doméstica.

## Metodologia

1. Coletaram-se dados diários de 8 moedas emergentes no período de janeiro de 2024 a março de 2025
2. As séries foram invertidas (para expressar unidades locais por USD) e normalizadas em base 100
3. Estimaram-se os pesos do controle sintético via otimização por mínimos quadrados com restrição de não-negatividade (w ≥ 0), utilizando apenas o período pré-evento como base de treinamento
4. Comparou-se a trajetória do BRL real com o controle sintético antes e após 27/11/2024
5. Conduziram-se testes de robustez: teste placebo, teste t de Welch e análise de R²

## Evento

**Data:** 27 de novembro de 2024  
**Declaração:** Proposta de isenção de Imposto de Renda para salários de até R$ 5.000/mês  
**Contexto:** A notícia gerou forte reação nos mercados financeiros, com depreciação imediata do real frente ao dólar, suscitando o debate sobre o impacto fiscal esperado da medida.

## Bloco 1 — Instalação e Importação de Pacotes

In [ ]:
# Instalação dos pacotes necessários
!pip install yfinance cvxpy --upgrade --quiet
!pip install pandas numpy matplotlib scikit-learn scipy --quiet

In [ ]:
# Importação das bibliotecas
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import ttest_ind

# Parâmetros globais da análise
DATA_INICIO = '2024-01-01'
DATA_FIM    = '2025-03-11'
DATA_EVENTO = pd.to_datetime('2024-11-27')

## Bloco 2 — Coleta e Tratamento dos Dados

Coletaram-se cotações diárias de 8 moedas emergentes via Yahoo Finance. A seleção buscou representar economias com perfil de sensibilidade semelhante ao Brasil em relação ao ciclo global de commodities e ao apetite internacional por risco: México (MXN), Chile (CLP), Colômbia (COP), África do Sul (ZAR), Turquia (TRY), Índia (INR) e Indonésia (IDR).

As séries foram invertidas para expressar a moeda local por unidade de USD — convenção na qual depreciação corresponde a alta —, e normalizadas em base 100 a partir de 01/01/2024.

In [ ]:
# Definição dos tickers das moedas emergentes
tickers = {
    'BRL': 'BRL=X',
    'MXN': 'MXN=X',
    'CLP': 'CLP=X',
    'COP': 'COP=X',
    'ZAR': 'ZAR=X',
    'TRY': 'TRY=X',
    'INR': 'INR=X',
    'IDR': 'IDR=X',
}

# Download dos preços de fechamento
dados_raw = yf.download(
    list(tickers.values()),
    start=DATA_INICIO,
    end=DATA_FIM,
    progress=False
)['Close']
dados_raw.columns = list(tickers.keys())

# Correção de spike na TRY em 27/12/2024 (artefato de dado identificado na série)
dados_raw.loc['2024-12-27', 'TRY'] = np.nan
dados_raw['TRY'] = dados_raw['TRY'].ffill()

# Inversão das séries: Yahoo Finance retorna USD/XYZ; inverteu-se para XYZ/USD
dados_invertido = 1 / dados_raw

# Normalização em base 100 a partir da primeira observação disponível
dados_base = dados_invertido / dados_invertido.iloc[0] * 100

# Remoção de datas com valores ausentes (feriados locais das diferentes praças)
dados_base = dados_base.dropna()

print(f'Período coberto: {dados_base.index[0].date()} a {dados_base.index[-1].date()}')
print(f'Total de observações: {len(dados_base)}')
dados_base.head()

## Bloco 3 — Construção do Controle Sintético

O BRL sintético foi construído como combinação linear ponderada das moedas doadoras. Os pesos foram estimados via otimização quadrática com restrição de não-negatividade (w ≥ 0), treinada exclusivamente sobre o período anterior ao evento.

Optou-se pela restrição de pesos positivos para garantir interpretabilidade econômica: cada moeda doadora contribui positivamente para a cesta sintética, sem posições vendidas implícitas.

In [ ]:
# Separação entre BRL (variável tratada) e moedas doadoras
brl_real = dados_base['BRL']
doadores = dados_base.drop(columns='BRL')

# Subconjunto de treinamento: período pré-evento exclusivamente
X_treino = doadores.loc[doadores.index < DATA_EVENTO]
y_treino = brl_real.loc[brl_real.index < DATA_EVENTO]

# Otimização: minimização do erro quadrático com restrição w >= 0
w          = cp.Variable(doadores.shape[1])
objective  = cp.Minimize(cp.sum_squares(X_treino.values @ w - y_treino.values))
prob       = cp.Problem(objective, [w >= 0])
prob.solve()

# Armazenamento dos pesos estimados
pesos = pd.Series(w.value, index=doadores.columns)

# Construção da série sintética para o período completo
brl_sintetico = doadores @ pesos

# Exibição dos pesos com contribuição relevante (> 1%)
print('Pesos estimados para o controle sintético (contribuição > 1%):')
print(pesos[pesos > 0.01].sort_values(ascending=False).round(4))

## Bloco 4 — Validação do Ajuste Pré-Evento

Avaliou-se a qualidade do ajuste do controle sintético no período anterior ao evento. Um R² elevado indica rastreamento adequado do BRL antes da fala, conferindo credibilidade à hipótese de que o desvio pós-evento é atribuível ao choque doméstico e não a falhas do modelo.

In [ ]:
# Avaliação das métricas de ajuste no período de treinamento
sint_pre = brl_sintetico.loc[brl_sintetico.index < DATA_EVENTO]
r2       = r2_score(y_treino, sint_pre)
mse_pre  = mean_squared_error(y_treino, sint_pre)

print(f'R² pré-evento  : {r2:.4f}')
print(f'MSE pré-evento : {mse_pre:.4f}')

if r2 >= 0.90:
    print('=> Ajuste considerado excelente (R² >= 0.90)')
elif r2 >= 0.75:
    print('=> Ajuste considerado satisfatório (R² >= 0.75)')
else:
    print('=> Ajuste abaixo do esperado — resultados devem ser interpretados com cautela')

## Bloco 5 — Visualização Principal

Comparou-se a trajetória do BRL real com o controle sintético ao longo de todo o período analisado. A linha vertical vermelha indica a data do evento. O distanciamento entre as curvas após 27/11/2024 representa o desvio atribuível ao choque doméstico, controlado pelo movimento das demais moedas emergentes.

In [ ]:
# Gráfico principal: BRL real vs. BRL sintético em base 100
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(brl_real,      label='BRL Real',      linewidth=2,   color='steelblue')
ax.plot(brl_sintetico, label='BRL Sintético', linewidth=2,   color='orange', linestyle='--')
ax.axvline(DATA_EVENTO, color='red', linestyle=':', linewidth=1.5,
           label='Fala do Ministro Haddad (27/11/2024)')

ax.set_title('BRL Real vs. Controle Sintético (base 100 = jan/2024)', fontsize=13)
ax.set_xlabel('Data')
ax.set_ylabel('Índice (base 100)')
ax.legend()
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

In [ ]:
# Conversão para cotação nominal USD/BRL
valor_inicial         = dados_invertido['BRL'].iloc[0]
brl_real_cotacao      = brl_real      * (valor_inicial / 100)
brl_sintetico_cotacao = brl_sintetico * (valor_inicial / 100)

# Inversão para convenção de mercado: USD/BRL
usdbrl_real      = 1 / brl_real_cotacao
usdbrl_sintetico = 1 / brl_sintetico_cotacao

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(usdbrl_real,      label='USD/BRL Real',      linewidth=2, color='steelblue')
ax.plot(usdbrl_sintetico, label='USD/BRL Sintético', linewidth=2, color='orange', linestyle='--')
ax.axvline(DATA_EVENTO, color='red', linestyle=':', linewidth=1.5,
           label='Fala do Ministro Haddad (27/11/2024)')

ax.set_title('USD/BRL Real vs. Controle Sintético (cotação nominal)', fontsize=13)
ax.set_xlabel('Data')
ax.set_ylabel('R$/USD')
ax.legend()
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

## Bloco 6 — Mensuração do Efeito

Calculou-se o gap entre o BRL real e o sintético na data do evento e ao longo dos dias subsequentes. O gap representa, em termos da base 100, o excesso de depreciação atribuível ao choque doméstico após controle pelo comportamento das demais moedas emergentes.

In [ ]:
# Cálculo do gap diário
gap_diario = brl_real - brl_sintetico

# Métricas de impacto
gap_evento     = gap_diario.loc[DATA_EVENTO]
gap_evento_pct = gap_evento / brl_sintetico.loc[DATA_EVENTO] * 100
gap_medio_pre  = gap_diario[gap_diario.index <  DATA_EVENTO].mean()
gap_medio_pos  = gap_diario[gap_diario.index >  DATA_EVENTO].mean()
dias_equilibrio = (gap_diario[gap_diario.index > DATA_EVENTO].abs() <= 0.5).sum()

print(f'Gap absoluto em {DATA_EVENTO.date()}  : {gap_evento:.2f} pontos (base 100)')
print(f'Gap percentual em {DATA_EVENTO.date()}: {gap_evento_pct:.2f}%')
print(f'Gap médio pré-evento                  : {gap_medio_pre:.2f}')
print(f'Gap médio pós-evento                  : {gap_medio_pos:.2f}')
print(f'Dias de retorno ao equilíbrio (gap <= 0.5): {dias_equilibrio}')

In [ ]:
# Evolução do gap diário
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Painel superior: gap diário
axes[0].plot(gap_diario, color='darkblue', label='Gap diário (BRL - Sintético)')
axes[0].axvline(DATA_EVENTO, color='red', linestyle=':', linewidth=1.5,
                label='Fala do Ministro Haddad')
axes[0].axhline(0, color='gray', linestyle='--', linewidth=1)
axes[0].set_title('Desvio Diário entre BRL Real e Controle Sintético', fontsize=12)
axes[0].set_ylabel('Desvio (pontos base 100)')
axes[0].legend()
axes[0].grid(True, alpha=0.4)

# Painel inferior: gap acumulado pós-evento
gap_acumulado = gap_diario[gap_diario.index > DATA_EVENTO].cumsum()
axes[1].plot(gap_acumulado, color='darkred', label='Gap acumulado pós-evento')
axes[1].axhline(0, color='gray', linestyle='--', linewidth=1)
axes[1].set_title('Acúmulo de Desvio do BRL em Relação ao Controle Sintético (pós-evento)', fontsize=12)
axes[1].set_ylabel('Pontos acumulados (base 100)')
axes[1].set_xlabel('Data')
axes[1].legend()
axes[1].grid(True, alpha=0.4)

plt.tight_layout()
plt.show()

## Bloco 7 — Testes de Robustez

### 7.1 Teste Placebo

Aplicou-se o mesmo procedimento de controle sintético a cada uma das moedas doadoras, tratando-as individualmente como 'moeda tratada'. Espera-se que o gap do BRL seja significativamente superior ao das demais moedas — caso contrário, o efeito identificado poderia ser atribuído ao método e não ao evento em si.

### 7.2 Teste t de Welch

Verificou-se estatisticamente se o gap médio pós-evento difere de forma significativa do gap médio pré-evento.

In [ ]:
# Teste placebo: aplicação do método a cada moeda doadora
resultados_placebo = {}

for moeda in doadores.columns:
    y_pl     = dados_base[moeda]
    X_pl     = dados_base.drop(columns=[moeda])
    X_pl_pre = X_pl.loc[X_pl.index < DATA_EVENTO]
    y_pl_pre = y_pl.loc[y_pl.index < DATA_EVENTO]

    w_pl   = cp.Variable(X_pl.shape[1])
    prob_pl = cp.Problem(
        cp.Minimize(cp.sum_squares(X_pl_pre.values @ w_pl - y_pl_pre.values)),
        [w_pl >= 0]
    )
    prob_pl.solve()

    sint_pl    = X_pl @ w_pl.value
    gap_pl_pct = (y_pl.loc[DATA_EVENTO] - sint_pl.loc[DATA_EVENTO]) / sint_pl.loc[DATA_EVENTO] * 100
    resultados_placebo[moeda] = gap_pl_pct

# Inclusão do BRL no comparativo
resultados_placebo['BRL'] = gap_evento_pct
placebo_series = pd.Series(resultados_placebo).sort_values(ascending=False)

print('Gap percentual na data do evento por moeda:')
print(placebo_series.round(2))

In [ ]:
# Visualização do teste placebo
cores = ['crimson' if m == 'BRL' else 'steelblue' for m in placebo_series.index]

fig, ax = plt.subplots(figsize=(10, 5))
placebo_series.plot(kind='bar', ax=ax, color=cores, edgecolor='white')
ax.axhline(0, color='gray', linestyle='--')
ax.set_title('Teste Placebo — Gap Percentual por Moeda na Data do Evento', fontsize=12)
ax.set_xlabel('Moeda')
ax.set_ylabel('Gap (%)')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Teste t de Welch: gap médio pré vs. pós evento
gap_pre_serie = gap_diario[gap_diario.index < DATA_EVENTO]
gap_pos_serie = gap_diario[gap_diario.index > DATA_EVENTO]

t_stat, p_valor = ttest_ind(gap_pre_serie, gap_pos_serie, equal_var=False)

print('Teste t de Welch — Gap médio pré vs. pós evento')
print(f'Estatística t : {t_stat:.3f}')
print(f'p-valor       : {p_valor:.4f}')

if p_valor < 0.01:
    print('=> Diferença estatisticamente significativa ao nível de 1%')
elif p_valor < 0.05:
    print('=> Diferença estatisticamente significativa ao nível de 5%')
else:
    print('=> Diferença não significativa ao nível de 5%')

## Bloco 8 — Conclusões

Identificou-se um desvio positivo e persistente do BRL em relação ao controle sintético após 27/11/2024, sugerindo que a declaração sobre isenção de IRPF produziu depreciação adicional do real além do movimento observado nas demais moedas emergentes no mesmo período.

O teste placebo indicou que o gap do BRL foi superior ao de todas as moedas doadoras na data do evento, reforçando a especificidade do choque doméstico. O teste t de Welch confirmou a significância estatística da diferença entre os gaps médios pré e pós-evento.

**Limitações:**
- O período de análise é relativamente curto (jan/2024 a mar/2025), o que pode limitar a estabilidade dos pesos estimados
- Outros eventos domésticos ocorridos simultaneamente — discussões fiscais e votações no Congresso — não foram isolados individualmente
- O controle sintético pressupõe que as moedas doadoras não foram afetadas pelo evento doméstico brasileiro — hipótese razoável, mas não testável diretamente

---
*Análise elaborada com fins educativos e de pesquisa. Não constitui recomendação de investimento.*